In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import zipfile
import os

In [4]:
# Unzip the zipped database file

# Path to your zip file
zip_path= './Data/dsc-phase-2-project-v3-main.zip'
##zip_path = './zippedData/im.db.zip'

# Extract to a specific directory (optional)
extract_path = './unzipped_files'

# Create the directory if it doesn't exist
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipping complete. Database file extracted to:", extract_path)

Unzipping complete. Database file extracted to: ./unzipped_files
